# 0.准备工作

## 0.1 导入数据集预处理、特征工程和模型训练所需的库

In [27]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import decomposition, ensemble

import pandas,  numpy, textblob, string #xgboost,
#from keras.preprocessing import text, sequence
#from keras import layers, models, optimizers

## 0.2 连接数据库并获取各类文本数据到列表
- 将词、诗、文言文、期刊和新闻的数据加入data列表中

In [28]:
import sqlite3
conn = sqlite3.connect("../data/database/texts.db")
print ('Opened database successfully')

Opened database successfully


In [29]:
c = conn.cursor()
ci_data=[]
cursor = c.execute("SELECT * from ci order by id asc limit 5000")
#cursor = c.execute("SELECT * from ci order by id")
for row in cursor:
    ci_data.append(row)
print("读取的词数据样例：")
print(ci_data[-2:])

poet_data = []
cursor=c.execute("SELECT * FROM poet order by id asc limit 5000")
for row in cursor:
    poet_data.append(row)
print("\n读取的诗数据样例：")
print(poet_data[-2:])

classical_data = []
cursor=c.execute("SELECT * FROM classical order by id asc limit 5000")
for row in cursor:
    classical_data.append(row)
print("\n读取的文言文数据样例：")
print(classical_data[-2:])

journal_data = []
cursor=c.execute("SELECT * FROM journal order by id asc limit 5000")
for row in cursor:
    journal_data.append(row)
print("\n读取的期刊数据样例：")
print(journal_data[:2])
    
news_data = []
cursor=c.execute("SELECT * FROM news order by id asc limit 5000")
for row in cursor:
    news_data.append(row)
print("\n读取的新闻数据样例：")
print(news_data[:2])

读取的词数据样例：
[(50005175, '真一长存，太虚同体，妙门自开。 既混元初判，两仪布景，复还根本，全籍灵台。 浩气冲开，谷神滋化，渐觉神光空际来。 幽绝处，听龙吟虎啸，蓦地风雷。 奇哉。 妙道难猜。 鲜点化、愚迷成大材。 试与君说破，分明状似，蚌含渊月，秋兔怀胎。 壮志男儿，当年高士，莫把身心惹世埃。 功成後，任身居紫府，名列仙阶。  ', 'ci', 5175), (50005176, '急急修行，细算人生，能有几时。 任万般千种风流好，柰一朝身死，不免抛离。 蓦地思量，死生事大，使我心如刀剑挥。 难留住，那金乌箭疾，玉兔梭飞。 早觉悟、莫教迟。 我清净、谁能婚少妻。 便假饶月里，娥见在，从他越国，有貌西施。 此个风流，更无心恋，且放宽怀免是非。 蓬莱路，仗三千行满，独跨鸾归。  ', 'ci', 5176)]

读取的诗数据样例：
[(40005227, '塔藏玉骨，云抱山腰。 迹尘外泯，道光内昭。 度金针之玉线，续凤弦之鸾胶。 壶春在而花芳枯木，夜鹤鸣而月浸孤巢。 家风清淡兮石牛饮水，儿孙秀拔兮天柱摩霄。 ', 'poet', 'ad19513a-ad0c-4c01-97d8-d030d05067c5'), (40005228, '凤眼鹤形，宗门伟匠。 量外提撕，声前敲唱。 据令兮长剑倚天，应机兮明珠在掌。 太虚有月兮老兔含霜，大海无风兮华鲸吹浪。 ', 'poet', '084a2ab5-4188-4b46-8e6f-2c9584ed2ff1')]

读取的文言文数据样例：
[(10006203, '\u3000\u3000凡发兵马，全指挥或三百人以上至五千人，用一虎一豹符；五千人以上，用双虎只豹符。以下符之法委枢密院，以右符第一为始，盛以木函，封以本院印，与宣命相副，付于使臣。宣内具言：下第一符发兵马若干。主将遇宣与符，即将左符看验得合，乃为兴发。发讫，即以本司印封题右符，还付使臣，归京，仍飞驿别奏。\n', 'classical', 'c_list_101.txt'), (10006204, '\u3000\u3000凡主将所掌符契，专择一官为腹心典领。\n', 'classical', 'c_list_101.txt')]

读取的期刊数据样例：
[(20000001, '作者程苏东, 北京大学中文系讲师 (北京100871) 。\n', 

## 0.3 建立拆分训练数据集
- 将数据库中text文本内容作为训练数据内容，type类型标签作为判断的依据，分别存入train_data和train_target两个列表中
- 拆分数据集，我们将测试集和训练集的切分比例为0.3

In [60]:
data = ci_data + poet_data + classical_data + journal_data + news_data

train_data=[]
train_target=[]
for i in range(0,len(data)):
    train_data.append(data[i][1])
    train_target.append(data[i][2])
print(train_data[:3])
print(train_target[15000:15003])

['气和玉烛，睿化著鸿明。 缇管一阳生。 郊盛礼燔柴毕，旋轸凤凰城。 森罗仪卫振华缨。 载路溢欢声。 皇图大业超前古，垂象泰阶平。 岁时丰衍，九土乐升平。 睹寰海澄清。 道高尧舜垂衣治，日月并文明。 嘉禾甘露登歌荐，云物焕祥经。 兢兢惕惕持谦德，未许禅云亭。', '严夜警，铜莲漏迟迟。 清禁肃，森陛戟，羽卫俨皇闱。 角声励，钲鼓攸宜。 金管成雅奏，逐吹逶迤。 荐苍璧，郊祀神祗。 属景运纯禧。 京坻丰衍，群材乐育，诸侯述职，盛德服蛮夷。 殊祥萃，九苞丹凤来仪。 膏露降，和气洽，三秀焕灵芝。 鸿猷播，史册相辉。 张四维。 卜世永固丕基。 敷玄化，荡荡无为。 合尧舜文思。 混并寰宇，休牛归马，销金偃革，蹈咏庆昌期。', '承宝运，驯致隆平。 鸿庆被寰瀛。 时清俗阜，治定功成。 遐迩咏由庚。 严郊祀，文物声明。 会天正、星拱奏严更。 布羽仪簪缨。 宸心虔洁，明德播惟馨。 动苍冥。 神降享精诚。 燔柴半，万乘移天仗，肃銮辂旋衡。 千官云拥，群后葵倾。 玉帛旅明庭。 韶荐，金奏谐声。 集休亨。 皇泽浃黎庶，普率洽恩荣。 仰钦元后，睿圣贯三灵。 万邦宁。 景贶福千龄。']
['modern', 'modern', 'modern']


In [45]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(train_data, train_target,test_size=0.3,random_state=0) 

#### 在前期摸索时，我们先使用的是将文段分成词语作为特征标识来进行机器学习
#### 而后又进行了将文段拆分成字并使用bigram来测度字间关系作为特征标识进行机器学习
- 下面先放上以“词语”为基础的训练部分

# 1、构建以词（word）为基础的训练数据集 
## 1.1 加入jieba分词
- 我们选用的是中文分词库jieba库来进行本次的分词处理，对于现代文（期刊论文+新闻）分词效果较好，对于古文（文言文、诗词）分词效果比较有歧义

In [32]:
import jieba
import numpy as np
import pandas as pd

def fenci(train_data):
    words_df = ' '.join(jieba.cut(train_data))
    return words_df

x_train_fenci = []
for i in range(len(x_train)):
    x_train_fenci.append(fenci(x_train[i]))
x_test_fenci = []
for i in range(len(x_test)):
    x_test_fenci.append(fenci(x_test[i]))
x_test_fenci[:1]

['\u3000 \u3000 武王 问 太公 曰 ： “ 论 将 之 道 奈何 ？ ” \u3000 \u3000 太公 曰 ： “ 将 有 五材 、 十过 。 ” \n']

## 1.2 引入停用词
* 停用词(Stop Words) 为“电脑检索中的虚字、非检索用字”，具体是指文本中出现频率很高，但实际意义又不大的词。
* 这一类主要包括了语气助词、副词、介词、连词等，通常自身并无明确意义，只有将其放入一个完整的句子中才有一定作用的词语。
* 停用词一定程度上相当于过滤词(Filter Words)
* 在本实验中我们小组下载了一份停用词表用于中文分词后的训练模型中以提高准确率

In [33]:
infile = open("../data/stopwords_cn.txt",encoding='utf-8')
stopwords_lst = infile.readlines()
stopwords = [x.strip() for x in stopwords_lst]
print(stopwords)

['的', '一', '不', '在', '人', '有', '是', '为', '以', '于', '上', '他', '而', '后', '之', '来', '及', '了', '因', '下', '可', '到', '由', '这', '与', '也', '此', '但', '并', '个', '其', '已', '无', '小', '我', '们', '起', '最', '再', '今', '去', '好', '只', '又', '或', '很', '亦', '某', '把', '那', '你', '乃', '它', '怎么', '任何', '连同', '开外', '再有', '哪些', '甚至于', '又及', '当然', '就是', '遵照', '以来', '赖以', '否则', '此间', '后者', '按照', '才是', '自身', '再则', '就算', '即便', '有些', '例如', '它们', '虽然', '为此', '以免', '别处', '我们', '依据', '趁着', '就要', '各位', '别的', '前者', '不外乎', '虽说', '除此', '个别', '的话', '甚而', '那般', '譬如', '作为', '谁人', '进而', '那边', '首先', '因此', '怎么样', '果然', '除非', '以上', '为何', '要么', '随时', '如果说', '诸如', '还是', '一旦', '基于', '本人', '因而', '继而', '不单', '此时', '等等', '截至', '不但', '故而', '全体', '从此', '对于', '朝着', '怎样', '以为', '那儿', '或是', '本身', '况且', '处在', '吧', '不至于', '那个', '被', '诸位', '从而', '比', '各自', '针对', '此外', '何处', '为了', '这般', '别', '仍旧', '既然', '反而', '关于', '较之', '不管', '趁', '彼时', '这边', '不光', '宁可', '要是', '其他', '其它', '由于', '还要', '经过', '不过', '来说', '当', '从', '除了', '到', '既是', '的确', '得', '说来', 


### 本次实验中我们在特征工程和模型构建时分别选择了多种方式
---
#### 特征工程：
1. 技术向量
2. TF-IDF（是否引入ngram参数以及analyzer参数共有3种特征）
---
#### 模型构建
1. 朴素贝叶斯模型
2. 线性回归模型
---
- 每一个构建的特征工程可以使用一个模型来训练，因此本次共有8组训练结果

# 2.特征工程

## 2.1 计数向量作为特征，以朴素贝叶斯为模型进行训练

- 计数向量是数据集的矩阵表示，其中每行代表来自语料库的文档，每列表示来自语料库的术语，并且每个单元格表示特定文档中特定术语的频率计数：

In [34]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}',stop_words=stopwords, max_features=5000)
vectorizer.fit_transform(x_train_fenci)

xtrain_count = vectorizer.transform(x_train_fenci)
xtest_count = vectorizer.transform(x_test_fenci)

- 词汇表示
- 通过构建计数向量，赋予每个词一个数值代表该词 

In [35]:
cv.vocabulary_

{'一生': 114,
 '政治': 2587,
 '学术': 1763,
 '都': 4627,
 '讲': 4231,
 '学派': 1765,
 '从未': 550,
 '后来': 1319,
 '写': 871,
 '中国': 320,
 '近': 4502,
 '近代': 4503,
 '分布': 960,
 '以后': 563,
 '历史': 1165,
 '哲学': 1383,
 '等': 3732,
 '均': 1503,
 '强调': 2133,
 '派': 3173,
 '研究': 3594,
 '重要': 4657,
 '进一步': 4524,
 '方法': 2667,
 '讲述': 4233,
 '文化': 2628,
 '及其': 1190,
 '做法': 736,
 '分为': 956,
 '史': 1270,
 '社会科学': 3616,
 '提出': 2527,
 '将': 1877,
 '各种': 1288,
 '分': 955,
 '为主': 345,
 '系': 3772,
 '所谓': 2361,
 '民族': 3088,
 '组织': 3809,
 '价值': 570,
 '权威': 2897,
 '世界': 271,
 '贡献': 4330,
 '继承': 3850,
 '做': 732,
 '整理': 2625,
 '解释': 4195,
 '工作': 1965,
 '外国': 1593,
 '思想': 2233,
 '融合': 4097,
 '第二': 3725,
 '面对': 4811,
 '集中': 4778,
 '抓住': 2411,
 '便': 675,
 '做到': 734,
 '主张': 362,
 '不同': 219,
 '看来': 3555,
 '多': 1600,
 '加上': 1048,
 '叙述': 1246,
 '第一': 3721,
 '事': 433,
 '以前': 562,
 '混合': 3232,
 '要': 4159,
 '讨论': 4221,
 '问题': 4714,
 '不可': 218,
 '风流': 4856,
 '强': 2127,
 '芳菲': 4022,
 '偷': 742,
 '叶': 1274,
 '霜': 4791,
 '秋': 3640,
 '道': 4591,


## 2.2 TF-IDF向量作为特征，以朴素贝叶斯为模型训练

- TF-IDF的分数代表了词语在文档和整个语料库中的相对重要性。
- TF-IDF分数由两部分组成：
>1. 第一部分是计算标准的词语频率（TF）
>2. 第二部分是逆文档频率（IDF）。
- 其中计算语料库中文档总数除以含有该词语的文档数量，然后再取对数就是逆文档频率。

- TF(t)=（该词语在文档出现的次数）/（文档中词语的总数）

- IDF(t)= log_e（文档总数/出现该词语的文档总数）
---

- TF-IDF向量可以由不同级别的分词产生（单个词语，词性，多个词（n-grams））
> 因此在本次以词语为特征的训练中我们分别以单个词语、多个词语ngram和词性分别为特征进行训练，因此一共得到3个训练模型


#### （1）词语级tf-idf：矩阵代表了每个词语在不同文档中的TF-IDF分数

In [36]:
#词语级tf-idf,使用tf-idf把文本转为向量
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}',stop_words=stopwords, max_features=7000, lowercase = False)
tfidf_vect.fit(x_train_fenci)
xtrain_tfidf = tfidf_vect.transform(x_train_fenci)
xtest_tfidf = tfidf_vect.transform(x_test_fenci)

#### （2）ngram 级tf-idf：N-grams是多个词语在一起的组合，这个矩阵代表了N-grams的TF-IDF分数。

In [37]:
# ngram 级tf-idf
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}',stop_words=stopwords, ngram_range=(1,2), max_features=7000, lowercase = False)
tfidf_vect_ngram.fit(x_train_fenci)
xtrain_tfidf_ngram = tfidf_vect_ngram.transform(x_train_fenci)
xtest_tfidf_ngram = tfidf_vect_ngram.transform(x_test_fenci)

#### （3）词性级tf-idf：矩阵代表了语料中多个词性的TF-IDF分数

In [38]:
#词性级tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(x_train_fenci)
xtrain_tfidf_ngram_chars = tfidf_vect_ngram_chars.transform(x_train_fenci)
xtest_tfidf_ngram_chars = tfidf_vect_ngram_chars.transform(x_test_fenci)

# 3、构建模型

### ·定义模型使用函数

In [39]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)

    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)

    if is_neural_net:
        predictions = predictions.argmax(axis=-1)

    return metrics.accuracy_score(predictions, y_test)

## 3.1 线性回归为模型对上述四个特征进行训练

In [40]:
# Linear Classifier on Count Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_count, y_train, xtest_count)
print("LR, Count Vectors: ", accuracy)

#特征为词语级别TF-IDF向量的线性分类器
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf, y_train, xtest_tfidf)
print("LR, WordLevel TF-IDF: ", accuracy)

#特征为多个词语级别TF-IDF向量的线性分类器
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram, y_train, xtest_tfidf_ngram)
print("LR, N-Gram Vectors: ", accuracy)

#特征为词性级别TF-IDF向量的线性分类器
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram_chars, y_train, xtest_tfidf_ngram_chars)
print("LR, CharLevel Vectors: ", accuracy)

C:\Users\jojo\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LR, Count Vectors:  0.8805333333333333


C:\Users\jojo\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LR, WordLevel TF-IDF:  0.8990666666666667


C:\Users\jojo\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LR, N-Gram Vectors:  0.8944
LR, CharLevel Vectors:  0.9636


C:\Users\jojo\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## 3.2 朴素贝叶斯模型训练

In [41]:
#特征为计数向量的朴素贝叶斯
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_count, y_train, xtest_count)
print("NB, Count Vectors: ", accuracy)

#特征为词语级别TF-IDF向量的朴素贝叶斯
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, y_train, xtest_tfidf)
print("NB, WordLevel TF-IDF: ", accuracy)

#特征为多个词语级别TF-IDF向量的朴素贝叶斯
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, y_train, xtest_tfidf_ngram)
print("NB, N-Gram Vectors: ", accuracy )

#特征为词性级别TF-IDF向量的朴素贝叶斯
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars, y_train, xtest_tfidf_ngram_chars)
print("NB, CharLevel Vectors: ", accuracy)

NB, Count Vectors:  0.8618666666666667
NB, WordLevel TF-IDF:  0.8784
NB, N-Gram Vectors:  0.872
NB, CharLevel Vectors:  0.9218666666666666


# 目前仍存在的问题：使用jieba分词对古文的分词效果存疑
- 因此下面我们小组将使用字间关系作为特征进行优化
---
> 由于单个字在文言文、诗词中区别不大，因此我们将舍弃只含有单个字的特征方程，只保留含有ngram的两个特征方程（计数向量ngram与TFIDF的ngram）

## 1、Basic方案：对原文本拆分成单字
### 暴力拆字

In [42]:
data_list = []
for line in data:
    data_list.append(list(line))

print(data_list[6000])
for item in data_list:
    item[1] = ' '.join(list(item[1]))
print(data_list[6000])
print(len(data_list))

[40001139, '独上江楼望故乡，泪襟霜笛共凄凉。 云生陇首秋虽早，月在天心夜已长。 魂梦只能随蛱蝶，烟波无计学鸳鸯。 蜀笺都有三千幅，总写离情寄孟光。 ', 'poet', 'a3e2bff9-e5a5-4f79-ad94-663be75d15f5']
[40001139, '独 上 江 楼 望 故 乡 ， 泪 襟 霜 笛 共 凄 凉 。   云 生 陇 首 秋 虽 早 ， 月 在 天 心 夜 已 长 。   魂 梦 只 能 随 蛱 蝶 ， 烟 波 无 计 学 鸳 鸯 。   蜀 笺 都 有 三 千 幅 ， 总 写 离 情 寄 孟 光 。  ', 'poet', 'a3e2bff9-e5a5-4f79-ad94-663be75d15f5']
25000


- 建立训练集、测试集

In [48]:
data_list = []
for line in data:
    data_list.append(list(line))

for item in data_list:
    item[1] = ' '.join(list(item[1]))
    
print(data_list[6000])
print(len(data_list))

[40001139, '独 上 江 楼 望 故 乡 ， 泪 襟 霜 笛 共 凄 凉 。   云 生 陇 首 秋 虽 早 ， 月 在 天 心 夜 已 长 。   魂 梦 只 能 随 蛱 蝶 ， 烟 波 无 计 学 鸳 鸯 。   蜀 笺 都 有 三 千 幅 ， 总 写 离 情 寄 孟 光 。  ', 'poet', 'a3e2bff9-e5a5-4f79-ad94-663be75d15f5']
25000


In [49]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(train_data, train_target,test_size=0.3,random_state=0)

- 分别构建计数向量和TFIDF特征方程

In [55]:
# ngram 级计数向量
vectorizer = CountVectorizer(analyzer='char',ngram_range=(1,2), max_features=5000)
vector = vectorizer.fit_transform(x_train)
vector.todense()

matrix([[ 0,  1, 39, ...,  0,  0,  0],
        [ 0,  0,  4, ...,  0,  0,  0],
        [ 0,  1,  0, ...,  0,  0,  0],
        ...,
        [ 0,  0,  2, ...,  0,  0,  0],
        [ 0,  1,  2, ...,  0,  0,  0],
        [ 0,  0,  7, ...,  0,  0,  0]], dtype=int64)

In [56]:
vectorizer.vocabulary_

{'梁': 2748,
 '启': 1336,
 '超': 4087,
 '一': 302,
 '生': 3133,
 ',': 103,
 ' ': 2,
 '无': 2469,
 '论': 3947,
 '在': 1446,
 '政': 2396,
 '治': 2880,
 '还': 4172,
 '是': 2520,
 '学': 1692,
 '术': 2642,
 '上': 353,
 '都': 4285,
 '流': 2921,
 '质': 4034,
 '变': 1245,
 '而': 3660,
 '好': 1632,
 '讲': 3944,
 '派': 2919,
 '却': 1195,
 '始': 1650,
 '终': 3568,
 '贯': 4041,
 '从': 656,
 '未': 2632,
 '动': 1108,
 '摇': 2374,
 '。': 204,
 '他': 660,
 '后': 1315,
 '来': 2674,
 '写': 949,
 '《': 291,
 '清': 2959,
 '代': 668,
 '概': 2764,
 '》': 292,
 '、': 190,
 '中': 449,
 '国': 1423,
 '近': 4167,
 '三': 348,
 '百': 3193,
 '年': 1936,
 '史': 1279,
 '风': 4506,
 '之': 502,
 '地': 1467,
 '理': 3109,
 '的': 3196,
 '分': 1011,
 '布': 1905,
 '老': 3645,
 '孔': 1679,
 '墨': 1537,
 '以': 674,
 '观': 3887,
 '历': 1200,
 '研': 3391,
 '究': 3457,
 '法': 2886,
 '(': 99,
 '补': 3849,
 '编': 3610,
 ')': 101,
 '儒': 850,
 '家': 1754,
 '哲': 1384,
 '等': 3494,
 '著': 3804,
 '作': 766,
 '均': 1482,
 '强': 2033,
 '调': 4000,
 '于': 581,
 '重': 4309,
 '要': 3876,
 '并': 1950,
 '进': 4189,
 '步'

In [57]:
# ngram 级tf-idf
tfidf_vect_ngram = TfidfVectorizer(analyzer='char', ngram_range=(1,2), max_features=5000, lowercase = False)
tfidf_vect_ngram.fit(x_train)

TfidfVectorizer(analyzer='char', lowercase=False, max_features=5000,
                ngram_range=(1, 2))

In [ ]:
print("暴力拆字方法的训练结果：")


## 2、Pro: 清理时去掉符号（利用正则表达式）

In [59]:
import re

train_data_pro = []
for paragraph in train_data:
    train_data_pro.append(' '.join(list((re.sub("[\s+\.\!\/_,$%^*)(+\"\']+|[+——！，。？、~@#￥%……&*（）]", "",paragraph)))))


print("去除符号成功：\n样例：\n【"+train_data[6000]+"】\n=========>\n【"+train_data_pro[6000]+"】")

去除符号成功：
样例：
【独上江楼望故乡，泪襟霜笛共凄凉。 云生陇首秋虽早，月在天心夜已长。 魂梦只能随蛱蝶，烟波无计学鸳鸯。 蜀笺都有三千幅，总写离情寄孟光。 】
=========>
【独 上 江 楼 望 故 乡 泪 襟 霜 笛 共 凄 凉 云 生 陇 首 秋 虽 早 月 在 天 心 夜 已 长 魂 梦 只 能 随 蛱 蝶 烟 波 无 计 学 鸳 鸯 蜀 笺 都 有 三 千 幅 总 写 离 情 寄 孟 光】


In [ ]:
x_train_pro, x_test_pro, y_train, y_test = train_test_split(train_data_pro, train_target,test_size=0.3,random_state=0)

cv = CountVectorizer(analyzer='char',)
vector_pro = cv.fit_transform(x_train_pro)
vector_pro.todense()
vectorizer_pro = CountVectorizer(analyzer='char', max_features=5000)
vectorizer_pro.fit(x_train_pro)
print(x_train_pro[:3])

## 3、Pro+: 头尾加上`<start>` `<end>` 标记

In [ ]:
train_data_prop = []
for paragraph in train_data_pro:
    train_data_prop.append("<start> "+paragraph+" <end>")

print(train_data_prop[6000])

In [ ]:
x_train_prop, x_test_prop, y_train, y_test = train_test_split(train_data_prop, train_target,test_size=0.3,random_state=0)
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(analyzer='char')
vector_prop = cv.fit_transform(x_train_prop)
vector_prop.todense()
vectorizer_prop = CountVectorizer(analyzer='char', max_features=5000)
vectorizer_prop.fit(x_train_prop)
print(x_train_prop[4000])